# PostgreSQL Connection and Analysis Notebook

This notebook provides functionality to connect to PostgreSQL using the configuration from `config_boomer_load.yml` and run queries with results returned as pandas DataFrames.

## Environment Setup

First, let's ensure all required packages are installed in the current kernel environment.

In [3]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package):
    """Install a package using pip in the current kernel environment."""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installed successfully")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error installing {package}: {e}")

# Required packages
required_packages = [
    "psycopg2-binary==2.9.7",
    "pandas",
    "PyYAML==6.0.1"
]

print("Installing required packages...")
for package in required_packages:
    install_package(package)

print("\n" + "="*50)
print("Package installation completed!")
print("="*50)

Installing required packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 2.1 MB/s eta 0:00:00a 0:00:01
✅ psycopg2-binary==2.9.7 installed successfully
✅ pandas installed successfully
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2
✅ PyYAML==6.0.1 installed successfully

Package installation completed!


In [4]:
# Import required libraries with error handling
try:
    import pandas as pd
    print("✅ pandas imported successfully")
except ImportError as e:
    print(f"❌ Error importing pandas: {e}")

try:
    import psycopg2
    print(f"✅ psycopg2 imported successfully (version: {psycopg2.__version__})")
except ImportError as e:
    print(f"❌ Error importing psycopg2: {e}")
    print("Please run the installation cell above first!")

try:
    import yaml
    print("✅ yaml imported successfully")
except ImportError as e:
    print(f"❌ Error importing yaml: {e}")

import os
import sys
from typing import Optional, Dict, Any

# Add src directory to path for imports
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

print("\n" + "="*50)
print("All libraries imported successfully!")
print("="*50)

✅ pandas imported successfully
✅ psycopg2 imported successfully (version: 2.9.7 (dt dec pq3 ext lo64))
✅ yaml imported successfully

All libraries imported successfully!


In [6]:
def load_config(config_path: str = '../config/config_boomer_load.yml') -> Dict[str, Any]:
    """
    Load configuration from YAML file.
    
    Args:
        config_path: Path to the configuration file
        
    Returns:
        Dictionary containing configuration data
    """
    try:
        with open(config_path, 'r') as file:
            config = yaml.safe_load(file)
        print(f"Configuration loaded successfully from {config_path}")
        return config
    except FileNotFoundError:
        print(f"Configuration file not found: {config_path}")
        raise
    except yaml.YAMLError as e:
        print(f"Error parsing YAML file: {e}")
        raise

# Load configuration
config = load_config()
print("Available configuration sections:", list(config.keys()))

Configuration loaded successfully from ../config/config_boomer_load.yml
Available configuration sections: ['database', 'queries', 'neo4j_load_config']


In [7]:
def parse_postgres_config(config_string: str) -> Dict[str, str]:
    """
    Parse PostgreSQL configuration string into connection parameters.
    
    Args:
        config_string: Multi-line string containing PostgreSQL configuration
        
    Returns:
        Dictionary with connection parameters
    """
    db_config = {}
    
    for line in config_string.strip().split('\n'):
        if ':' in line:
            key, value = line.split(':', 1)
            db_config[key.strip()] = value.strip()
    
    return db_config

# Parse PostgreSQL configuration
postgres_config_str = config['database']['postgres']
postgres_config = parse_postgres_config(postgres_config_str)
print("PostgreSQL connection parameters:")
for key, value in postgres_config.items():
    if key == 'password':
        print(f"  {key}: {'*' * len(value)}")
    else:
        print(f"  {key}: {value}")

PostgreSQL connection parameters:
  host: 4.150.184.135
  port: 5432
  database: SERP_TREND_DEV_DB
  user: serptrend_user
  password: ***********************


In [8]:
def execute_postgres_query(query: str, config: Dict[str, str]) -> pd.DataFrame:
    """
    Execute a PostgreSQL query and return results as a pandas DataFrame.
    
    Args:
        query: SQL query to execute
        config: PostgreSQL connection configuration
        
    Returns:
        pandas DataFrame containing query results
    """
    connection = None
    try:
        # Establish connection
        connection = psycopg2.connect(
            host=config['host'],
            port=config['port'],
            database=config['database'],
            user=config['user'],
            password=config['password']
        )
        
        print(f"Connected to PostgreSQL database: {config['database']}")
        
        # Execute query and return as DataFrame
        df = pd.read_sql_query(query, connection)
        
        print(f"Query executed successfully. Retrieved {len(df)} rows.")
        return df
        
    except psycopg2.Error as e:
        print(f"PostgreSQL error: {e}")
        raise
    except Exception as e:
        print(f"Unexpected error: {e}")
        raise
    finally:
        if connection:
            connection.close()
            print("Database connection closed.")

print("PostgreSQL query function defined successfully!")

PostgreSQL query function defined successfully!


In [15]:
# Test the connection and function with the sample query
sample_query = "SELECT * FROM structured_content LIMIT 2;"

print(f"Executing sample query: {sample_query}")
print("-" * 50)

try:
    result_df = execute_postgres_query(sample_query, postgres_config)
    
    print("\nQuery Results:")
    print(f"Shape: {result_df.shape}")
    print(f"Columns: {list(result_df.columns)}")
    print("\nFirst few rows:")
    display(result_df.head())
    
except Exception as e:
    print(f"Error executing query: {e}")

Executing sample query: SELECT * FROM structured_content LIMIT 2;
--------------------------------------------------
Connected to PostgreSQL database: SERP_TREND_DEV_DB


/var/folders/ct/w3w0zg9d5w10mhf7h_d_xwym0000gn/T/ipykernel_53797/3822124451.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Query executed successfully. Retrieved 2 rows.
Database connection closed.

Query Results:
Shape: (2, 18)
Columns: ['id', 'url', 'raw_html_id', 'domain', 'site_name', 'title', 'author', 'publish_date', 'content', 'summary', 'tags', 'language', 'word_count', 'meta_description', 'extracted_at', 'is_latest', 'run_id', 'status_neo4j']

First few rows:


,id,url,raw_html_id,domain,site_name,title,author,publish_date,content,summary,tags,language,word_count,meta_description,extracted_at,is_latest,run_id,status_neo4j
0,36,https://www.kff.org/medicaid/issue-brief/healt...,37,kff.org,None,Health Provisions in the 2025 Federal Budget ...,None,None,None,This summary describes the health care provisi...,"[Medicare, Medicaid, Affordable Care Act, Heal...",en,None,None,2025-08-11 10:18:44.264541+00:00,True,935ad96d-6fcf-4aa4-a045-f778afdbe6ee,False
1,37,https://oig.hhs.gov/,38,hhs.gov,None,Office of Inspector General | Government Overs...,None,None,None,A majority of the OIG's resources goes toward ...,"[Medicare, Medicaid, government oversight, fraud]",en,None,None,2025-08-11 10:18:45.257664+00:00,True,935ad96d-6fcf-4aa4-a045-f778afdbe6ee,False


In [16]:
result_df.columns

Index(['id', 'url', 'raw_html_id', 'domain', 'site_name', 'title', 'author',
       'publish_date', 'content', 'summary', 'tags', 'language', 'word_count',
       'meta_description', 'extracted_at', 'is_latest', 'run_id',
       'status_neo4j'],
      dtype='object')

In [18]:
result_df = execute_postgres_query("SELECT count(*) FROM structured_content;",postgres_config)
result_df

Connected to PostgreSQL database: SERP_TREND_DEV_DB


/var/folders/ct/w3w0zg9d5w10mhf7h_d_xwym0000gn/T/ipykernel_53797/3822124451.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Query executed successfully. Retrieved 1 rows.
Database connection closed.


,count
0,33


## Custom Query Execution

Use the cell below to execute your own custom queries. Simply modify the `custom_query` variable and run the cell.

In [ ]:
# Define your custom query here
custom_query = """
SELECT 
    COUNT(*) as total_records,
    COUNT(DISTINCT id) as unique_ids
FROM structured_content;
"""

print(f"Executing custom query: {custom_query.strip()}")
print("-" * 50)

try:
    custom_result = execute_postgres_query(custom_query, postgres_config)
    
    print("\nCustom Query Results:")
    display(custom_result)
    
except Exception as e:
    print(f"Error executing custom query: {e}")

## Data Analysis Examples

Here are some example analyses you can perform with the data:

In [ ]:
# Example: Get table schema information
schema_query = """
SELECT 
    column_name,
    data_type,
    is_nullable,
    column_default
FROM information_schema.columns 
WHERE table_name = 'structured_content'
ORDER BY ordinal_position;
"""

try:
    schema_df = execute_postgres_query(schema_query, postgres_config)
    print("\nTable Schema:")
    display(schema_df)
except Exception as e:
    print(f"Error getting schema: {e}")

In [ ]:
# Example: Basic data exploration
exploration_query = """
SELECT 
    COUNT(*) as total_rows,
    COUNT(DISTINCT id) as unique_ids,
    MIN(created_at) as earliest_date,
    MAX(created_at) as latest_date
FROM structured_content;
"""

try:
    exploration_df = execute_postgres_query(exploration_query, postgres_config)
    print("\nData Exploration:")
    display(exploration_df)
except Exception as e:
    print(f"Error in data exploration: {e}")

## Alternative Installation Methods

If the automatic installation above doesn't work, try these alternative methods:

In [ ]:
# Alternative 1: Install from requirements.txt
# Run this cell if you want to install all project dependencies
import subprocess
import sys

try:
    result = subprocess.run([sys.executable, "-m", "pip", "install", "-r", "../requirements.txt"], 
                          capture_output=True, text=True, check=True)
    print("✅ All requirements installed successfully")
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"❌ Error installing requirements: {e}")
    print(f"Error output: {e.stderr}")

In [ ]:
# Alternative 2: Manual conda installation (if using conda environment)
# Uncomment and run if you're using conda

# import subprocess
# import sys

# try:
#     result = subprocess.run(["conda", "install", "-c", "conda-forge", "psycopg2", "-y"], 
#                           capture_output=True, text=True, check=True)
#     print("✅ psycopg2 installed via conda successfully")
#     print(result.stdout)
# except subprocess.CalledProcessError as e:
#     print(f"❌ Error installing via conda: {e}")
#     print(f"Error output: {e.stderr}")

## Quick Reference

### Main Function
```python
# Execute any PostgreSQL query and get results as DataFrame
df = execute_postgres_query("YOUR_SQL_QUERY_HERE", postgres_config)
```

### Configuration
- Configuration is loaded from `../config/config_boomer_load.yml`
- PostgreSQL connection parameters are automatically parsed
- Connection is automatically closed after each query

### Error Handling
- All database errors are caught and displayed
- Connection is always properly closed
- Detailed error messages help with troubleshooting

### Installation Troubleshooting
If you encounter issues with psycopg2 installation:

1. **Try psycopg2-binary**: This is usually easier to install
2. **Check your Python environment**: Make sure you're in the correct virtual environment
3. **System dependencies**: On some systems, you may need to install PostgreSQL development headers
4. **Use conda**: If using Anaconda/Miniconda, try installing via conda instead of pip

### Command Line Installation
If the notebook installation fails, try these commands in your terminal:

```bash
# Using pip
pip install psycopg2-binary==2.9.7

# Using conda
conda install -c conda-forge psycopg2

# Install all project requirements
pip install -r requirements.txt
```